In [3]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import os

# Momentum Investing

Momentum investing involves a strategy to capitalize on the continuance of an existing market trend. ([Source: Investopedia](https://www.investopedia.com/terms/m/momentum_investing.asp))

In [4]:
# change the directiory to root to allow importing of py files saved in algorithmictrading
os.chdir(r'..\\')
from algorithmictrading.secrets import IEX_CLOUD_API_TOKEN

# Data Loading - S&P 500 Index
The S&P 500 Index is one of the most common benchmarks for US Large Cap stocks. It tracks the performance of 500 of the largest companies in the United States.

You can substitute any list of tickers for this equal weight walk-through. The list of stocks should be aved in the `\data` folder.

In [6]:
stocks = pd.read_csv(r'.\data\sp_500_stocks.csv')
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


# Connecting to the IEX API
We will be using the free IEX Cloud API for the market data. Data is purposefully scrambled and is NOT meant for production!

[Documentation can be found here.](https://iexcloud.io/docs/api/#testing-sandbox)

We can use the base URL and concatenate a string from the API IEX documentation in order to pull the data.
We can pass the following into the string for a specific data request:
- `symbol`
- `token`

In [8]:
BASE_URL = 'https://sandbox.iexapis.com/stable'
symbol = 'AAPL'
stats = f'/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(BASE_URL+stats).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2173625669763,
 'week52high': 141.36,
 'week52low': 58.01,
 'week52change': 0.6933281163316503,
 'sharesOutstanding': 17462085410,
 'float': 0,
 'avg10Volume': 119688057,
 'avg30Volume': 117945720,
 'day200MovingAvg': 117.92,
 'day50MovingAvg': 129.95,
 'employees': 0,
 'ttmEPS': 3.29,
 'ttmDividendRate': 0.8400082494886704,
 'dividendYield': 0.006490665779938774,
 'nextDividendDate': '0',
 'exDividendDate': '2020-11-05',
 'nextEarningsDate': '0',
 'peRatio': 39.505951256412,
 'beta': 1.1720825462923594,
 'maxChangePercent': 48.9134602448249,
 'year5ChangePercent': 4.707256853794466,
 'year2ChangePercent': 2.506256873581805,
 'year1ChangePercent': 0.688802859169871,
 'ytdChangePercent': -0.030602956595569525,
 'month6ChangePercent': 0.3523184306009466,
 'month3ChangePercent': 0.0384143316205799,
 'month1ChangePercent': 0.05351798877141296,
 'day30ChangePercent': 0.05224376040739527,
 'day5ChangePercent': -0.01694180525930723}

In [9]:
# momentum stat
data['year1ChangePercent']

0.688802859169871

# Making Batch API Calls
Making a single http request is really slow. We are much better served breaking up our security list into small batches. The IEX API limits 100 symbols per batch, so we we will make 6 http requests.

For our first **momentum** factor, will will use:
- `year1ChangePercent` - measures the absolute percentage performance of the share price over the past one year. It is to be contrasted with Relative Strength; it is calculated as (Current Price minus Old Price) divided by Old Price x 100 ([source: Stockopedia](https://www.stockopedia.com/ratios/price-change-over-last-year-35/))


In [10]:
def make_chunks(df):
     return np.array_split(df['Ticker'].to_list(), np.ceil(len(df) / 100))

def get_data_batch(df):
    df_list = []
    chunks = make_chunks(df)
    for chunk in chunks:
        ticker_strings = ','.join(chunk)
        batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={ticker_strings}&token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(batch_api_call_url).json()
        tickers = [k for k in data.keys()]
        latestprices = [data[k]['quote']['latestPrice'] for k in data.keys()]
        one_year_changes = [data[k]['stats']['year1ChangePercent'] for k in data.keys()]
        df = pd.DataFrame({'ticker': tickers, 'latest_price': latestprices, '1_year_change': one_year_changes})
        df_list.append(df)
    return  pd.concat(df_list, ignore_index=True)


In [11]:
df = get_data_batch(stocks)
df.head()

,ticker,latest_price,1_year_change
0,A,129.830,0.468475
1,AAL,15.865,-0.450572
2,AAP,174.940,0.200965
3,AAPL,129.750,0.682180
4,ABBV,113.080,0.303073


# Setting Stock Threshold and Keeping High Momentum Stocks

In [22]:
def transform_momentum_df(df, stock_cutoff=50):
    df = df.copy()
    return (df.sort_values('1_year_change', ascending=False)
              .reset_index(drop=True)
              .iloc[:stock_cutoff]
           )

In [25]:
mom_df = transform_momentum_df(df)
mom_df.head()

,ticker,latest_price,1_year_change
0,CARR,42.290,2.585607
1,ALB,182.680,1.620473
2,LB,47.567,1.468541
3,FCX,31.570,1.429277
4,NVDA,547.490,1.237922


In [26]:
def get_share_amounts(df, portfolio_size=50000000):
    share_amounts = portfolio_size / len(df.index)
    return df.assign(recommended_trades= lambda x: np.floor(share_amounts /  x['latest_price']))

In [27]:
final_df = get_share_amounts(mom_df)
final_df.head()

,ticker,latest_price,1_year_change,recommended_trades
0,CARR,42.290,2.585607,23646.0
1,ALB,182.680,1.620473,5474.0
2,LB,47.567,1.468541,21022.0
3,FCX,31.570,1.429277,31675.0
4,NVDA,547.490,1.237922,1826.0


# Improving On Our Momentum Strategy
We will introduce time horizons of outperformance in order to get a composite score of how high quality the increase in price has been.

All else being equal, we would prefer to have steadier price increase over time, as opposed to a more volatile price increase (perhaps due to specific news or world event).

The following factors will be extraced from IEX CLOUD:
- `1_year_change`
- `6_month_change`
- `3_month_change`
- `1_month_change`

In [28]:
def get_momentum_data_batch(df):
    df_list = []
    chunks = make_chunks(df)
    for chunk in chunks:
        ticker_strings = ','.join(chunk)
        batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={ticker_strings}&token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(batch_api_call_url).json()
        
        # get variables
        tickers = [k for k in data.keys()]
        latestprices = [data[k]['quote']['latestPrice'] for k in data.keys()]
        one_year_changes = [data[k]['stats']['year1ChangePercent'] for k in data.keys()]
        six_month_changes = [data[k]['stats']['month6ChangePercent'] for k in data.keys()]
        three_month_changes = [data[k]['stats']['month3ChangePercent'] for k in data.keys()]
        one_month_changes = [data[k]['stats']['month1ChangePercent'] for k in data.keys()]
        df = pd.DataFrame({'ticker': tickers, 
                           'latest_price': latestprices, 
                           '1_year_change': one_year_changes,
                           '6_month_change': six_month_changes,
                           '3_month_change': three_month_changes,
                           '1_month_change': one_month_changes
                          })
        df_list.append(df)
    return  pd.concat(df_list, ignore_index=True)

In [31]:
mom_df_2 = get_momentum_data_batch(stocks)
mom_df_2.head()

,ticker,latest_price,1_year_change,6_month_change,3_month_change,1_month_change
0,A,128.597,0.472273,0.431698,0.206514,0.074619
1,AAL,15.860,-0.451261,0.300810,0.190456,-0.098323
2,AAP,174.300,0.201175,0.320784,0.136987,0.125624
3,AAPL,136.765,0.684922,0.350474,0.038772,0.052681
4,ABBV,113.850,0.306633,0.171022,0.271605,0.033199


# Calculating Momentum Composite Score

We will now normalize each score and add a column called `hqm_score` (high quality momentum), which is the average of each time period momemtum variable.

- `set_index()` to move the columns we don't wish to rank to the index
- apply the `rank(pct=True)` function to calculate the percentile rank of each momentum metric
- `assign` a new column, `hqm_score`, which takes the column-wise mean of the normalized momentum variables
- `reset_index()` to reset the dataframe
- take the first 50 stock names and return the final dataframe
- call the `get_share_amounts` function to calculate how many shares to buy in an equally-weighted momentum-driven portfolio

In [74]:
def generate_high_quality_momentum_score(df, stock_cutoff=50):
    return (df.copy()
              .set_index(['ticker', 'latest_price'])
              .rank(pct=True)
              .assign(hqm_score= lambda x: x.mean(axis='columns'))
              .sort_values('hqm_score', ascending=False)
              .reset_index()
              .head(stock_cutoff)
           )

In [75]:
mom_df_2_pct = generate_high_quality_momentum_score(mom_df_2)
mom_df_2_pct.head()

,ticker,latest_price,1_year_change,6_month_change,3_month_change,1_month_change,hqm_score
0,ALB,180.40,0.998004,0.990020,0.986028,0.998004,0.993014
1,FCX,31.45,0.994012,0.996008,0.984032,0.984032,0.989521
2,SIVB,445.51,0.972056,0.984032,0.960080,0.988024,0.976048
3,LB,47.77,0.996008,1.000000,0.910180,0.952096,0.964571
4,APTV,149.49,0.954092,0.974052,0.940120,0.976048,0.961078


In [77]:
# call get_share_amounts from earlier in the notebook
final_df2 = get_share_amounts(mom_df_2_pct)
final_df2.head()

,ticker,latest_price,1_year_change,6_month_change,3_month_change,1_month_change,hqm_score,recommended_trades
0,ALB,180.40,0.998004,0.990020,0.986028,0.998004,0.993014,5543.0
1,FCX,31.45,0.994012,0.996008,0.984032,0.984032,0.989521,31796.0
2,SIVB,445.51,0.972056,0.984032,0.960080,0.988024,0.976048,2244.0
3,LB,47.77,0.996008,1.000000,0.910180,0.952096,0.964571,20933.0
4,APTV,149.49,0.954092,0.974052,0.940120,0.976048,0.961078,6689.0


# Exporting Data to Excel

Pandas can easily output to a csv file of xlsx file natively. However, if we want to output to a styled xlsx file, we can use `xlsxwriter` to customize the output to a much greater degree. 

In [81]:
writer = pd.ExcelWriter(r'.\data\momentum_recommended_trades.xlsx', engine='xlsxwriter')
final_df2.to_excel(writer, sheet_name='Recommended Trades', index = False)

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'border_color': font_color
        }
    )

dollar_format = writer.book.add_format(
        {
        'num_format':'$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'border_color': font_color
        }
    )

pct_format = writer.book.add_format(
        {
        'num_format':'0.00%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'border_color': font_color
        }
    )

float_format = writer.book.add_format(
        {
        'num_format':'0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'border_color': font_color
        }
    )

integer_format = writer.book.add_format(
        {
        'num_format':'0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'border_color': font_color
        }
    )


column_formats = { 
                    'A': ['ticker', string_format],
                    'B': ['latest_price', dollar_format],
                    'C': ['1_year_change', pct_format],
                    'D': ['6_month_change', pct_format],
                    'E': ['3_month_change', pct_format],
                    'F': ['1_month_change', pct_format],
                    'G': ['hqm_score', float_format],
                    'H': ['recommended_trades', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)
    
writer.sheets['Recommended Trades'].hide_gridlines(2)

writer.save()